In [ ]:
# Importing the essential libraries
import mne
import neurokit2 as nk
import pandas as pd
import numpy as np
from scipy import signal
import antropy as ant
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

# SHANNON ENTROPY AND N-GRAMS

In [ ]:
# shannon entropy and n-gram extraction from microstate sequences of subject 1

# reading the dataset
eeg_file_raw = r"C:\Users\90537\Desktop\sub-001_task-eyesclosed_eeg.set"
raw_eeg = mne.io.read_raw_eeglab(eeg_file_raw, preload=True)

shannon_entropies = []
ngrams = []

# defining the segment length and the number of microstates
segment_len = 5
number_of_microstates = 4

for time in range(int(raw_eeg.n_times / raw_eeg.info["sfreq"] / segment_len)):
    start_time = time * segment_len
    end_time = (time + 1) * segment_len

    # segmentation (5 seconds) using 4 microstates
    microstates = nk.microstates_segment(raw_eeg.copy().crop(tmin=start_time, tmax=end_time), n_microstates=number_of_microstates, random_state=42)

		# shannon entropy
    shannon_entropy = nk.microstates_complexity(microstates, show=False)

		# n-grams
    sequence = microstates["Sequence"]
    all_pairs = [f"{i}{j}" for i in range(number_of_microstates) for j in range(number_of_microstates)]
    pair_counts = {pair: 0 for pair in all_pairs}
    for i in range(len(sequence) - 1):
        pair = f"{int(sequence[i])}{int(sequence[i + 1])}"
        pair_counts[pair] += 1
    pair_counts = {pair: count / (len(sequence) - 1) for pair, count in pair_counts.items()}
    n_gram = pd.DataFrame({str(key): [value] for key, value in sorted(pair_counts.items())})
    n_gram = pd.DataFrame(n_gram)

    shannon_entropies.append(shannon_entropy)
    ngrams.append(n_gram)

# feature matrix construction
shannon_entropy_df = pd.concat(shannon_entropies, ignore_index=True)
ngrams_df = pd.concat(ngrams, ignore_index=True)

subject1_features_df = pd.merge(shannon_entropy_df, ngrams_df, left_index=True, right_index=True, how="inner")
# labels (AD=0, FTD=1, CN=2)
subject1_features_df["label"] = 0
subject1_features_df

,Microstates_Entropy_Shannon,00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33,label
0,1.967081,0.1628,0.0064,0.0120,0.0040,0.0068,0.1988,0.0108,0.0080,0.0104,0.0112,0.2236,0.0140,0.0048,0.0080,0.0128,0.3056,0
1,1.732418,0.4092,0.0204,0.0072,0.0088,0.0172,0.2872,0.0064,0.0080,0.0076,0.0048,0.0460,0.0028,0.0116,0.0064,0.0020,0.1544,0
2,1.876482,0.2480,0.0180,0.0056,0.0124,0.0168,0.3512,0.0080,0.0048,0.0048,0.0072,0.0892,0.0048,0.0148,0.0044,0.0028,0.2072,0
3,1.728390,0.1500,0.0164,0.0036,0.0032,0.0132,0.3624,0.0192,0.0024,0.0072,0.0172,0.3468,0.0032,0.0028,0.0012,0.0048,0.0464,0
4,1.941501,0.2084,0.0064,0.0032,0.0140,0.0052,0.1624,0.0088,0.0068,0.0024,0.0080,0.1896,0.0104,0.0160,0.0064,0.0092,0.3428,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,1.917783,0.1184,0.0048,0.0020,0.0036,0.0040,0.2652,0.0152,0.0076,0.0032,0.0132,0.3220,0.0096,0.0032,0.0088,0.0088,0.2104,0
115,1.935958,0.2796,0.0012,0.0124,0.0164,0.0016,0.2048,0.0032,0.0140,0.0104,0.0056,0.1260,0.0016,0.0180,0.0120,0.0024,0.2908,0
116,1.892679,0.3532,0.0180,0.0072,0.0020,0.0172,0.2596,0.0012,0.0152,0.0080,0.0020,0.1128,0.0056,0.0020,0.0136,0.0076,0.1748,0
117,1.890390,0.3608,0.0052,0.0048,0.0140,0.0040,0.1072,0.0044,0.0040,0.0064,0.0048,0.2032,0.0088,0.0136,0.0024,0.0108,0.2456,0


In [ ]:
df = pd.read_csv(r"C:\Users\90537\Desktop\eeg_microstate_features.csv")
df

,Microstates_Entropy_Shannon,00,01,02,03,10,11,12,13,20,...,22,23,30,31,32,33,subject_id,group,label,mmse
0,1.967081,0.1628,0.0064,0.0120,0.0040,0.0068,0.1988,0.0108,0.0080,0.0104,...,0.2236,0.0140,0.0048,0.0080,0.0128,0.3056,sub-001,A,0,16
1,1.732418,0.4092,0.0204,0.0072,0.0088,0.0172,0.2872,0.0064,0.0080,0.0076,...,0.0460,0.0028,0.0116,0.0064,0.0020,0.1544,sub-001,A,0,16
2,1.876482,0.2480,0.0180,0.0056,0.0124,0.0168,0.3512,0.0080,0.0048,0.0048,...,0.0892,0.0048,0.0148,0.0044,0.0028,0.2072,sub-001,A,0,16
3,1.728390,0.1500,0.0164,0.0036,0.0032,0.0132,0.3624,0.0192,0.0024,0.0072,...,0.3468,0.0032,0.0028,0.0012,0.0048,0.0464,sub-001,A,0,16
4,1.941501,0.2084,0.0064,0.0032,0.0140,0.0052,0.1624,0.0088,0.0068,0.0024,...,0.1896,0.0104,0.0160,0.0064,0.0092,0.3428,sub-001,A,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13914,1.930741,0.1944,0.0016,0.0160,0.0040,0.0020,0.2216,0.0172,0.0104,0.0140,...,0.3388,0.0032,0.0056,0.0096,0.0020,0.1416,sub-088,F,1,24
13915,1.957401,0.1672,0.0020,0.0072,0.0104,0.0028,0.1816,0.0096,0.0068,0.0100,...,0.3104,0.0108,0.0072,0.0072,0.0136,0.2436,sub-088,F,1,24
13916,1.712166,0.2428,0.0184,0.0064,0.0012,0.0216,0.4760,0.0020,0.0080,0.0040,...,0.0940,0.0056,0.0004,0.0108,0.0040,0.1020,sub-088,F,1,24
13917,1.747420,0.1100,0.0052,0.0056,0.0044,0.0056,0.0832,0.0028,0.0044,0.0060,...,0.3064,0.0184,0.0036,0.0056,0.0176,0.4192,sub-088,F,1,24


In [ ]:
def compute_accuracy(df, groups, label_col="label", drop_cols=None, test_size=0.3, random_state=42):
    if drop_cols is None:
        drop_cols = ["Segment", "subject_id", "task", "Gender", "Age", "Group", "MMSE"]
    
    df_sub = df[df["Group"].isin(groups)].copy()
    df_sub = df_sub.drop(columns=drop_cols)

    X = df_sub.drop(label_col, axis=1)
    y = df_sub[label_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    model = RandomForestClassifier(random_state=random_state)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# Azheimer's and control group
accuracy1 = compute_accuracy(df, ["A", "C"])
accuracy1

0.5962323390894819

In [ ]:
# Alzhemier's and Frontotemporal Dementia groups
accuracy2 = compute_accuracy(df, ["A", "F"])
accuracy2

0.6389904901243599

In [ ]:
# Frontotemporal dementia and control group
accuracy3 = compute_accuracy(df, ["C", "F"])
accuracy3

0.6462612982744453

# BAND POWER

In [ ]:
# band power extraction

# Loading the data
eeg_file_raw = r"C:\Users\90537\Desktop\sub-001_task-eyesclosed_eeg.set"
raw_eeg = mne.io.read_raw_eeglab(eeg_file_raw, preload=True)
eeg_data = raw_eeg.pick_types(eeg=True, eog=False)
eeg_values, times = eeg_data[:]
eeg_df = pd.DataFrame(eeg_values.T, columns = eeg_data.info['ch_names'])
eeg_df = eeg_df * 1e6
eeg_df['Segment'] = (eeg_df.index // 2500) + 1

# Extracting band power for each 5-second long segment and channel
def band_power(data, fs, segment_len=5):
    frequency_bands = {
        'DELTA': [0.5, 3.5],
        'THETA': [4, 7],
        'ALPHA': [8, 13],
        'BETA': [13, 25],
        'GAMMA': [25, 48]
    }

    results = {}

    for column in data.columns:
        x = data[column].values
        frequencies, psd = signal.welch(x, fs=fs, nperseg=fs*segment_len)

        for band, freq_range in frequency_bands.items():
            freq_indices = np.where((frequencies >= freq_range[0]) & (frequencies <= freq_range[1]))
            power = np.trapz(psd[freq_indices], frequencies[freq_indices])
            results[f"{band}_{column}"] = power

    return pd.DataFrame([results])

segment_band_powers = []

fs = raw_eeg.info['sfreq']

for segment in eeg_df['Segment'].unique():
    segment_data = eeg_df[eeg_df['Segment'] == segment].drop(columns=['Segment'])
    band_powers = band_power(segment_data, fs)
    segment_band_powers.append(band_powers)

band_power_extracted_df = pd.concat(segment_band_powers, ignore_index=True)
band_power_extracted_df

,DELTA_Fp1,THETA_Fp1,ALPHA_Fp1,BETA_Fp1,GAMMA_Fp1,DELTA_Fp2,THETA_Fp2,ALPHA_Fp2,BETA_Fp2,GAMMA_Fp2,...,DELTA_Cz,THETA_Cz,ALPHA_Cz,BETA_Cz,GAMMA_Cz,DELTA_Pz,THETA_Pz,ALPHA_Pz,BETA_Pz,GAMMA_Pz
0,284.697084,18.913390,12.984215,15.471460,21.528973,312.553518,23.730304,11.840599,13.074867,14.578874,...,306.193402,27.395848,10.799345,9.726889,4.618204,292.499440,24.151958,7.897336,10.563118,3.947215
1,306.220565,30.759718,12.743909,14.015082,23.111478,288.624596,35.480601,12.217555,12.478809,17.374728,...,260.147535,35.746060,9.717523,9.504930,4.390667,274.208810,38.222855,11.698157,8.575558,3.783892
2,502.324619,42.217642,17.092462,13.061131,19.519356,527.090862,41.791542,15.898147,10.516679,11.774964,...,501.118733,22.022181,7.391783,8.261102,3.368433,521.199883,23.258088,8.111669,9.446482,3.326713
3,261.824073,60.010358,10.346113,12.508530,14.191157,269.259232,48.582221,10.157338,9.401466,11.514445,...,281.426823,22.654651,8.789954,9.305889,4.032210,262.228310,18.319705,9.027466,7.305243,3.153441
4,664.592802,50.171178,17.822465,32.989858,36.952963,500.234376,48.672910,12.237329,19.186098,15.749492,...,349.816758,39.314292,8.540658,7.949960,3.038881,359.136678,31.171242,10.579283,8.829851,4.628694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,491.012400,39.042609,16.100209,12.434983,16.597286,498.135491,44.773672,14.138106,10.086645,10.196931,...,515.946040,35.998784,10.632019,8.217261,3.977112,539.943939,39.072592,12.833901,6.281385,3.125628
116,308.178654,35.863135,19.530473,9.379759,10.924663,321.814472,35.404765,21.899539,7.819388,6.668184,...,289.032612,37.425343,16.221302,6.756418,4.421358,337.677845,36.549072,12.998184,5.511381,3.528895
117,576.911706,35.646191,11.646534,8.777193,14.419847,601.517223,38.430148,13.735543,7.418935,9.382768,...,554.984989,25.638489,10.908309,5.847682,4.867397,510.927702,40.144715,11.058726,6.235502,3.219916
118,524.545439,54.801651,11.107350,10.575734,12.211616,526.339535,42.079828,11.616655,8.589573,7.154430,...,489.066120,41.309670,9.068504,6.450998,3.723714,448.713658,49.406728,7.426146,6.508032,3.824007


In [ ]:
df = pd.read_csv(r"C:\Users\90537\Desktop\all_subjects_bandpower.csv")
df['label'] = df['Group'].map({'A': 0, 'F': 1, 'C': 2})
df

,DELTA_Fp1,THETA_Fp1,ALPHA_Fp1,BETA_Fp1,GAMMA_Fp1,DELTA_Fp2,THETA_Fp2,ALPHA_Fp2,BETA_Fp2,GAMMA_Fp2,...,THETA_Pz,ALPHA_Pz,BETA_Pz,GAMMA_Pz,subject_id,task,Gender,Age,Group,MMSE
0,284.697084,18.913390,12.984215,15.471460,21.528973,312.553518,23.730304,11.840599,13.074867,14.578874,...,24.151958,7.897336,10.563118,3.947215,sub-001,eyesclosed,F,57,A,16
1,306.220565,30.759718,12.743909,14.015082,23.111478,288.624596,35.480601,12.217555,12.478809,17.374728,...,38.222855,11.698157,8.575558,3.783892,sub-001,eyesclosed,F,57,A,16
2,502.324619,42.217642,17.092462,13.061131,19.519356,527.090862,41.791542,15.898147,10.516679,11.774964,...,23.258088,8.111669,9.446482,3.326713,sub-001,eyesclosed,F,57,A,16
3,261.824073,60.010358,10.346113,12.508530,14.191157,269.259232,48.582221,10.157338,9.401466,11.514445,...,18.319705,9.027466,7.305243,3.153441,sub-001,eyesclosed,F,57,A,16
4,664.592802,50.171178,17.822465,32.989858,36.952963,500.234376,48.672910,12.237329,19.186098,15.749492,...,31.171242,10.579283,8.829851,4.628694,sub-001,eyesclosed,F,57,A,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14002,476.501272,38.296160,15.444041,17.297410,5.043111,478.035878,37.050388,19.981912,18.378507,5.880209,...,33.033673,13.634733,13.632251,2.900793,sub-088,eyesclosed,M,55,F,24
14003,337.170636,50.032627,24.095559,20.840300,3.792492,330.027997,48.114172,26.029232,19.884081,4.095058,...,28.550665,15.570674,11.303642,3.045952,sub-088,eyesclosed,M,55,F,24
14004,390.611934,49.539589,23.975271,21.798118,4.001328,385.787809,52.922474,23.387685,24.344735,4.310276,...,21.850140,30.316613,13.652342,3.770310,sub-088,eyesclosed,M,55,F,24
14005,774.242903,45.661115,24.328200,14.219583,5.083355,775.943559,46.663320,25.060790,14.183223,5.856630,...,45.730660,10.644295,10.631992,3.115942,sub-088,eyesclosed,M,55,F,24


In [ ]:
def compute_accuracy(df, groups, label_col="label", drop_cols=None, test_size=0.3, random_state=42):
    if drop_cols is None:
        drop_cols = ["Segment", "subject_id", "task", "Gender", "Age", "Group", "MMSE"]
    
    df_sub = df[df["Group"].isin(groups)].copy()
    df_sub = df_sub.drop(columns=drop_cols)

    X = df_sub.drop(label_col, axis=1)
    y = df_sub[label_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    model = RandomForestClassifier(random_state=random_state)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# Azheimer's and control group
accuracy1 = compute_accuracy(df, ["A", "C"])
accuracy1

0.8801498127340824

In [ ]:
# Alzhemier's and Frontotemporal Dementia groups
accuracy2 = compute_accuracy(df, ["A", "F"])
accuracy2

0.8778625954198473

In [ ]:
# Frontotemporal dementia and control group
accuracy3 = compute_accuracy(df, ["C", "F"])
accuracy3

0.8991836734693878

# SINGULAR VALUE DECOMPOSITION (SVD) ENTROPY

In [ ]:
# singular value decomposition entropy extraction

# Loading the data
eeg_file_raw = r"C:\Users\90537\Desktop\sub-001_task-eyesclosed_eeg.set"
raw_eeg = mne.io.read_raw_eeglab(eeg_file_raw, preload=True)
eeg_data = raw_eeg.pick_types(eeg=True, eog=False)
eeg_values, times = eeg_data[:]
eeg_df = pd.DataFrame(eeg_values.T, columns = eeg_data.info['ch_names'])
eeg_df = eeg_df * 1e6
eeg_df['Segment'] = (eeg_df.index // 2500) + 1

# Extracting SVD entropy for each 5-second long segment and channel
electrodes = [col for col in eeg_df.columns if col != 'Segment']
segments = eeg_df['Segment'].unique()
column_names = [f"{electrode}_SVD_entropy" for electrode in electrodes]
entropy_results = pd.DataFrame(index=segments, columns=column_names)
for segment in segments:
    segment_data = eeg_df[eeg_df['Segment'] == segment]
    for electrode in electrodes:
        entropy_results.loc[segment, f"{electrode}_SVD_entropy"] = ant.svd_entropy(segment_data[electrode].values, normalize=True)
entropy_results = entropy_results.astype(float)
entropy_results

,Fp1_SVD_entropy,Fp2_SVD_entropy,F3_SVD_entropy,F4_SVD_entropy,C3_SVD_entropy,C4_SVD_entropy,P3_SVD_entropy,P4_SVD_entropy,O1_SVD_entropy,O2_SVD_entropy,F7_SVD_entropy,F8_SVD_entropy,T3_SVD_entropy,T4_SVD_entropy,T5_SVD_entropy,T6_SVD_entropy,Fz_SVD_entropy,Cz_SVD_entropy,Pz_SVD_entropy
1,0.193290,0.175791,0.146775,0.169337,0.134208,0.137817,0.131572,0.128658,0.145329,0.140873,0.149705,0.176458,0.151346,0.165963,0.152218,0.139279,0.136457,0.131308,0.129430
2,0.275876,0.253417,0.206632,0.196039,0.181523,0.175284,0.187438,0.174088,0.207411,0.190922,0.212320,0.228126,0.200360,0.245106,0.227186,0.191709,0.196652,0.184730,0.177898
3,0.217937,0.186175,0.144739,0.135920,0.125919,0.137776,0.126706,0.133880,0.145632,0.140840,0.152926,0.160104,0.142126,0.175005,0.154774,0.142793,0.136164,0.128485,0.128391
4,0.267012,0.242030,0.194642,0.187065,0.179148,0.193350,0.184122,0.182918,0.200383,0.199626,0.201399,0.213560,0.204926,0.238953,0.233017,0.198077,0.192895,0.190548,0.182214
5,0.261617,0.226839,0.224444,0.190071,0.172720,0.176007,0.172187,0.178200,0.217394,0.224116,0.195105,0.201909,0.199135,0.221691,0.271365,0.199944,0.190470,0.177966,0.172396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.193389,0.170513,0.155331,0.164810,0.142081,0.135502,0.138791,0.137934,0.160546,0.151138,0.149636,0.170011,0.166023,0.153495,0.169566,0.158631,0.145005,0.140142,0.134812
117,0.186371,0.165520,0.152632,0.171226,0.142865,0.141009,0.145325,0.145415,0.173571,0.157938,0.149473,0.159501,0.163344,0.149243,0.182148,0.158697,0.141712,0.153897,0.144417
118,0.189181,0.170404,0.156292,0.176799,0.134827,0.142176,0.137484,0.146615,0.165495,0.154739,0.159806,0.173940,0.172226,0.158114,0.178876,0.160894,0.145769,0.149746,0.140869
119,0.192901,0.165364,0.155215,0.165165,0.138872,0.142083,0.141382,0.147332,0.162242,0.150593,0.147601,0.167378,0.158954,0.153069,0.172128,0.157513,0.142493,0.140462,0.145101


In [ ]:
df = pd.read_csv(r"C:\Users\90537\Desktop\all_subjects_svd_entropy.csv")
df['label'] = df['Group'].map({'A': 0, 'F': 1, 'C': 2})
df

,Segment,Fp1_SVD_entropy,Fp2_SVD_entropy,F3_SVD_entropy,F4_SVD_entropy,C3_SVD_entropy,C4_SVD_entropy,P3_SVD_entropy,P4_SVD_entropy,O1_SVD_entropy,...,Fz_SVD_entropy,Cz_SVD_entropy,Pz_SVD_entropy,subject_id,task,Gender,Age,Group,MMSE,label
0,1,0.193290,0.175791,0.146775,0.169337,0.134208,0.137817,0.131572,0.128658,0.145329,...,0.136457,0.131308,0.129430,sub-001,eyesclosed,F,57,A,16,0
1,2,0.275876,0.253417,0.206632,0.196039,0.181523,0.175284,0.187438,0.174088,0.207411,...,0.196652,0.184730,0.177898,sub-001,eyesclosed,F,57,A,16,0
2,3,0.217937,0.186175,0.144739,0.135920,0.125919,0.137776,0.126706,0.133880,0.145632,...,0.136164,0.128485,0.128391,sub-001,eyesclosed,F,57,A,16,0
3,4,0.267012,0.242030,0.194642,0.187065,0.179148,0.193350,0.184122,0.182918,0.200383,...,0.192895,0.190548,0.182214,sub-001,eyesclosed,F,57,A,16,0
4,5,0.261617,0.226839,0.224444,0.190071,0.172720,0.176007,0.172187,0.178200,0.217394,...,0.190470,0.177966,0.172396,sub-001,eyesclosed,F,57,A,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14002,153,0.151358,0.157390,0.148511,0.145939,0.138928,0.141872,0.143550,0.152777,0.163509,...,0.150263,0.138603,0.143989,sub-088,eyesclosed,M,55,F,24,1
14003,154,0.149345,0.156195,0.144628,0.146558,0.133958,0.136742,0.137386,0.139163,0.156797,...,0.147370,0.129351,0.129093,sub-088,eyesclosed,M,55,F,24,1
14004,155,0.174668,0.179036,0.168846,0.170800,0.155547,0.152099,0.162979,0.164893,0.185956,...,0.174376,0.157325,0.162497,sub-088,eyesclosed,M,55,F,24,1
14005,156,0.172969,0.177136,0.168180,0.166119,0.150217,0.145781,0.154926,0.156438,0.170738,...,0.170845,0.153336,0.150727,sub-088,eyesclosed,M,55,F,24,1


In [ ]:
def compute_accuracy(df, groups, label_col="label", drop_cols=None, test_size=0.3, random_state=42):
    if drop_cols is None:
        drop_cols = ["Segment", "subject_id", "task", "Gender", "Age", "Group", "MMSE"]
    
    df_sub = df[df["Group"].isin(groups)].copy()
    df_sub = df_sub.drop(columns=drop_cols)

    X = df_sub.drop(label_col, axis=1)
    y = df_sub[label_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    model = RandomForestClassifier(random_state=random_state)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# Azheimer's and control group
accuracy1 = compute_accuracy(df, ["A", "C"])
accuracy1

0.8439450686641697

In [ ]:
# Alzhemier's and Frontotemporal Dementia groups
accuracy2 = compute_accuracy(df, ["A", "F"])
accuracy2

0.8578698655034533

In [ ]:
# Frontotemporal dementia and control group
# Frontotemporal dementia and control group
accuracy3 = compute_accuracy(df, ["C", "F"])
accuracy3

0.8893877551020408

# HJORT PARAMETERS

In [ ]:
# hjort parameters (mobility and complexity) extraction

# Loading the data
eeg_file_raw = r"C:\Users\90537\Desktop\sub-001_task-eyesclosed_eeg.set"
raw_eeg = mne.io.read_raw_eeglab(eeg_file_raw, preload=True)
eeg_data = raw_eeg.pick_types(eeg=True, eog=False)
eeg_values, times = eeg_data[:]
eeg_df = pd.DataFrame(eeg_values.T, columns = eeg_data.info['ch_names'])
eeg_df = eeg_df * 1e6
eeg_df['Segment'] = (eeg_df.index // 2500) + 1
eeg_df

# Extracting hjort mobility and complexity for each 5-second long segment and channel
electrodes = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
              'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
segments = eeg_df['Segment'].unique()
column_names = [f"{electrode}_Hjorth_mobility" for electrode in electrodes]
entropy_results1 = pd.DataFrame(index=segments, columns=column_names)
for segment in segments:
    segment_data = eeg_df[eeg_df['Segment'] == segment]
    for electrode in electrodes:
        entropy_results1.loc[segment, f"{electrode}_Hjorth_mobility"] = ant.hjorth_params(segment_data[electrode].values)[0]
electrodes = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
              'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
segments = eeg_df['Segment'].unique()
column_names = [f"{electrode}_Hjorth_complexity" for electrode in electrodes]
entropy_results2 = pd.DataFrame(index=segments, columns=column_names)
for segment in segments:
    segment_data = eeg_df[eeg_df['Segment'] == segment]
    for electrode in electrodes:
        entropy_results2.loc[segment, f"{electrode}_Hjorth_complexity"] = ant.hjorth_params(segment_data[electrode].values)[1]
merged_df = pd.concat([entropy_results1, entropy_results2], axis=1)
merged_df

,Fp1_Hjorth_mobility,Fp2_Hjorth_mobility,F3_Hjorth_mobility,F4_Hjorth_mobility,C3_Hjorth_mobility,C4_Hjorth_mobility,P3_Hjorth_mobility,P4_Hjorth_mobility,O1_Hjorth_mobility,O2_Hjorth_mobility,...,O2_Hjorth_complexity,F7_Hjorth_complexity,F8_Hjorth_complexity,T3_Hjorth_complexity,T4_Hjorth_complexity,T5_Hjorth_complexity,T6_Hjorth_complexity,Fz_Hjorth_complexity,Cz_Hjorth_complexity,Pz_Hjorth_complexity
1,0.057649,0.050747,0.040215,0.048188,0.03613,0.037307,0.035387,0.034496,0.040028,0.038649,...,8.546488,8.872628,7.570707,8.853817,8.437077,8.438026,8.501729,9.163717,9.071617,8.877521
2,0.095492,0.084879,0.064649,0.060636,0.054677,0.052142,0.057035,0.052222,0.065275,0.05863,...,5.360897,5.520641,4.946664,5.672217,4.880542,4.981015,5.248863,5.466389,5.542677,5.589308
3,0.068077,0.055008,0.039829,0.036874,0.033578,0.037424,0.03374,0.036192,0.04002,0.038459,...,8.960793,8.511272,8.125494,8.909678,7.879276,8.707629,8.529115,8.943685,9.24725,9.481512
4,0.091326,0.079555,0.059422,0.056791,0.053515,0.05896,0.055597,0.055104,0.061795,0.061752,...,5.47151,5.723806,5.444255,5.701604,5.136644,4.965489,5.382186,5.773119,5.713072,5.839306
5,0.088723,0.072703,0.071709,0.057409,0.051123,0.051826,0.050803,0.052969,0.068428,0.072062,...,5.025921,6.088503,6.296022,5.799218,5.336324,4.295793,5.59445,5.854867,6.087517,6.072271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.058801,0.049804,0.044207,0.04756,0.039645,0.037272,0.03857,0.038207,0.046108,0.042726,...,7.978889,8.180091,7.35374,7.875519,7.948452,7.504812,7.646458,7.93869,8.198623,8.424887
117,0.055616,0.047625,0.042956,0.049556,0.039842,0.039091,0.040676,0.040614,0.050768,0.044909,...,7.535388,7.805332,7.693883,7.523403,8.066687,6.768474,7.356849,7.810969,7.37,7.531884
118,0.057112,0.049949,0.044532,0.052074,0.036958,0.039435,0.037846,0.041048,0.047885,0.043931,...,7.830705,7.519486,7.062693,7.250396,7.800906,6.954141,7.735702,8.247027,8.082966,8.313939
119,0.057679,0.04723,0.043629,0.047133,0.038228,0.039205,0.039225,0.041036,0.04655,0.042256,...,7.727241,8.055665,7.81813,7.528614,7.995153,6.963293,7.545454,8.315099,8.110574,7.733048


In [ ]:
df = pd.read_csv(r"C:\Users\90537\Desktop\all_subjects_hjort_parameters.csv")
df['label'] = df['Group'].map({'A': 0, 'F': 1, 'C': 2})
df

,Segment,Fp1_Hjorth_mobility,Fp2_Hjorth_mobility,F3_Hjorth_mobility,F4_Hjorth_mobility,C3_Hjorth_mobility,C4_Hjorth_mobility,P3_Hjorth_mobility,P4_Hjorth_mobility,O1_Hjorth_mobility,...,Fz_Hjorth_complexity,Cz_Hjorth_complexity,Pz_Hjorth_complexity,subject_id,task,Gender,Age,Group,MMSE,label
0,1,0.057649,0.050747,0.040215,0.048188,0.036130,0.037307,0.035387,0.034496,0.040028,...,9.163717,9.071617,8.877521,sub-001,eyesclosed,F,57,A,16,0
1,2,0.095492,0.084879,0.064649,0.060636,0.054677,0.052142,0.057035,0.052222,0.065275,...,5.466389,5.542677,5.589308,sub-001,eyesclosed,F,57,A,16,0
2,3,0.068077,0.055008,0.039829,0.036874,0.033578,0.037424,0.033740,0.036192,0.040020,...,8.943685,9.247250,9.481512,sub-001,eyesclosed,F,57,A,16,0
3,4,0.091326,0.079555,0.059422,0.056791,0.053515,0.058960,0.055597,0.055104,0.061795,...,5.773119,5.713072,5.839306,sub-001,eyesclosed,F,57,A,16,0
4,5,0.088723,0.072703,0.071709,0.057409,0.051123,0.051826,0.050803,0.052969,0.068428,...,5.854867,6.087517,6.072271,sub-001,eyesclosed,F,57,A,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14002,153,0.043485,0.045669,0.042418,0.041668,0.038911,0.040241,0.040527,0.044340,0.048190,...,6.642057,7.513925,6.943809,sub-088,eyesclosed,M,55,F,24,1
14003,154,0.043554,0.045970,0.041696,0.042406,0.037716,0.038670,0.038803,0.039565,0.046218,...,6.651289,8.265626,8.182219,sub-088,eyesclosed,M,55,F,24,1
14004,155,0.052587,0.054290,0.050246,0.050769,0.044845,0.043502,0.047845,0.048821,0.057680,...,5.550139,6.565186,5.960032,sub-088,eyesclosed,M,55,F,24,1
14005,156,0.051408,0.053045,0.049658,0.048833,0.042774,0.041139,0.044729,0.045322,0.051076,...,5.825090,6.940063,6.623316,sub-088,eyesclosed,M,55,F,24,1


In [ ]:
def compute_accuracy(df, groups, label_col="label", drop_cols=None, test_size=0.3, random_state=42):
    if drop_cols is None:
        drop_cols = ["Segment", "subject_id", "task", "Gender", "Age", "Group", "MMSE"]
    
    df_sub = df[df["Group"].isin(groups)].copy()
    df_sub = df_sub.drop(columns=drop_cols)

    X = df_sub.drop(label_col, axis=1)
    y = df_sub[label_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    model = RandomForestClassifier(random_state=random_state)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# Azheimer's and control group
accuracy1 = compute_accuracy(df, ["A", "C"])
accuracy1

0.8442571785268415

In [ ]:
# Alzhemier's and Frontotemporal Dementia groups
accuracy2 = compute_accuracy(df, ["A", "F"])
accuracy2

0.8600508905852418

In [ ]:
# Frontotemporal dementia and control group
accuracy3 = compute_accuracy(df, ["C", "F"])
accuracy3

0.8848979591836734